# MLOps con Databricks: Guía Completa

## 📚 Objetivos de Aprendizaje

En este notebook aprenderás:
1. **Desarrollo de modelos** con seguimiento automático
2. **MLflow** para experimentación y tracking
3. **Feature Store** para gestión de features
4. **Model Registry** para versionado y gobernanza
5. **Model Serving** para deployment en producción
6. **Monitoreo** de modelos en producción

---

## 🎯 Caso de Uso: Predicción de Churn en Telecomunicaciones

**Contexto de Negocio:**
Una empresa de telecomunicaciones necesita predecir qué clientes están en riesgo de cancelar su servicio (churn). El equipo de marketing puede entonces ofrecer incentivos personalizados para retenerlos.

**Requisitos MLOps:**
- Reentrenamiento mensual del modelo
- Monitoreo de performance en producción
- Versionado de modelos y features
- Deployment automatizado
- Trazabilidad completa

---

## 1️⃣ Setup Inicial y Configuración

In [ ]:
# Instalación de librerías necesarias
# En Databricks, muchas de estas librerías ya están preinstaladas
%pip install mlflow==2.10.0 databricks-feature-engineering scikit-learn==1.3.2
dbutils.library.restartPython()

In [ ]:
# Imports
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Databricks Feature Store
from databricks.feature_engineering import FeatureEngineeringClient

# ML Libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, 
    f1_score, roc_auc_score, confusion_matrix,
    classification_report
)

# Spark
from pyspark.sql import functions as F
from pyspark.sql.types import *

print("✅ Librerías importadas correctamente")
print(f"MLflow version: {mlflow.__version__}")

In [ ]:
# Configuración de MLflow
username = spark.sql("SELECT current_user()").first()[0]
experiment_name = f"/Users/{username}/mlops_telco_churn"

# Crear o obtener el experimento
mlflow.set_experiment(experiment_name)

# Configuración del Feature Store
fe = FeatureEngineeringClient()

# Nombres de catálogo y schema (usar Unity Catalog)
catalog_name = "main"  # Ajustar según tu configuración
schema_name = "mlops_demo"
database_name = f"{catalog_name}.{schema_name}"

# Crear schema si no existe
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {database_name}")

print(f"📊 Experimento: {experiment_name}")
print(f"📁 Database: {database_name}")

---

## 2️⃣ Generación de Datos Sintéticos

**📝 Nota Pedagógica:**
En producción, estos datos vendrían de tus fuentes reales (Data Lake, bases de datos, APIs). Aquí generamos datos sintéticos para simular un escenario realista.

In [ ]:
# Generar dataset sintético de clientes
np.random.seed(42)
n_customers = 5000

# IDs y datos demográficos
customer_ids = [f"CUST_{i:05d}" for i in range(n_customers)]
ages = np.random.randint(18, 75, n_customers)
tenure_months = np.random.randint(1, 72, n_customers)
monthly_charges = np.random.uniform(20, 120, n_customers)

# Servicios contratados
internet_service = np.random.choice(['DSL', 'Fiber', 'No'], n_customers)
has_phone = np.random.choice([0, 1], n_customers, p=[0.3, 0.7])
has_streaming = np.random.choice([0, 1], n_customers, p=[0.6, 0.4])
contract_type = np.random.choice(['Month-to-month', 'One year', 'Two year'], 
                                  n_customers, p=[0.5, 0.3, 0.2])

# Comportamiento de uso
monthly_minutes = np.random.randint(100, 2000, n_customers)
data_usage_gb = np.random.uniform(1, 50, n_customers)
support_tickets = np.random.poisson(lam=1.5, size=n_customers)
payment_method = np.random.choice(['Credit card', 'Bank transfer', 'Electronic check'],
                                   n_customers, p=[0.4, 0.3, 0.3])

# Generar churn con lógica de negocio
churn_probability = (
    0.05 +  # Base rate
    0.3 * (contract_type == 'Month-to-month') +
    0.2 * (tenure_months < 12) +
    0.15 * (support_tickets > 3) +
    0.1 * (payment_method == 'Electronic check') -
    0.1 * (has_streaming == 1)
)
churn = (np.random.random(n_customers) < churn_probability).astype(int)

# Crear DataFrame
data = pd.DataFrame({
    'customer_id': customer_ids,
    'age': ages,
    'tenure_months': tenure_months,
    'monthly_charges': monthly_charges,
    'internet_service': internet_service,
    'has_phone_service': has_phone,
    'has_streaming': has_streaming,
    'contract_type': contract_type,
    'monthly_minutes': monthly_minutes,
    'data_usage_gb': data_usage_gb,
    'support_tickets': support_tickets,
    'payment_method': payment_method,
    'churn': churn
})

print(f"✅ Dataset generado: {len(data):,} clientes")
print(f"📊 Tasa de churn: {churn.mean():.2%}")
data.head()

In [ ]:
# Análisis exploratorio rápido
print("\n📈 DISTRIBUCIÓN DE CHURN\n" + "="*50)
print(data.groupby('churn').size())

print("\n📊 ESTADÍSTICAS POR CONTRATO\n" + "="*50)
print(data.groupby('contract_type')['churn'].agg(['count', 'mean']))

print("\n🔍 CORRELACIÓN CON CHURN\n" + "="*50)
numeric_cols = ['age', 'tenure_months', 'monthly_charges', 'support_tickets']
correlations = data[numeric_cols + ['churn']].corr()['churn'].sort_values(ascending=False)
print(correlations)

---

## 3️⃣ Feature Store: Gestión Centralizada de Features

**🎯 ¿Por qué Feature Store?**
- **Reutilización**: Features compartidas entre equipos
- **Consistencia**: Mismas features en training y serving
- **Gobernanza**: Versionado y linaje de features
- **Performance**: Features pre-computadas

In [ ]:
# Convertir a Spark DataFrame
spark_df = spark.createDataFrame(data)

# Agregar timestamp (en producción vendría de tus datos)
spark_df = spark_df.withColumn(
    "update_timestamp",
    F.current_timestamp()
)

display(spark_df.limit(5))

In [ ]:
# FEATURE ENGINEERING: Crear features derivadas
features_df = spark_df.select(
    "customer_id",
    "update_timestamp",
    
    # Features base
    "age",
    "tenure_months",
    "monthly_charges",
    "monthly_minutes",
    "data_usage_gb",
    "support_tickets",
    
    # Features derivadas - Engagement
    (F.col("monthly_charges") / F.col("tenure_months")).alias("avg_monthly_spend"),
    (F.col("data_usage_gb") / 30).alias("daily_data_usage"),
    (F.col("support_tickets") / F.col("tenure_months")).alias("tickets_per_month"),
    
    # Features categóricas one-hot encoded
    F.when(F.col("internet_service") == "DSL", 1).otherwise(0).alias("is_dsl"),
    F.when(F.col("internet_service") == "Fiber", 1).otherwise(0).alias("is_fiber"),
    F.when(F.col("contract_type") == "Month-to-month", 1).otherwise(0).alias("is_monthly_contract"),
    F.when(F.col("contract_type") == "One year", 1).otherwise(0).alias("is_yearly_contract"),
    F.when(F.col("payment_method") == "Electronic check", 1).otherwise(0).alias("is_electronic_check"),
    
    # Servicios adicionales
    F.col("has_phone_service"),
    F.col("has_streaming"),
    
    # Segmentación de clientes
    F.when(F.col("tenure_months") < 6, "new")
     .when(F.col("tenure_months") < 24, "regular")
     .otherwise("loyal").alias("customer_segment"),
    
    # Target (solo para training)
    "churn"
)

print("✅ Features engineered creadas")
display(features_df.limit(5))

In [ ]:
# Crear Feature Table en Feature Store
feature_table_name = f"{database_name}.customer_churn_features"

try:
    # Crear la feature table
    fe.create_table(
        name=feature_table_name,
        primary_keys=["customer_id"],
        timestamp_keys=["update_timestamp"],
        df=features_df,
        description="Features de clientes para predicción de churn en telecomunicaciones"
    )
    print(f"✅ Feature table creada: {feature_table_name}")
    
except Exception as e:
    if "already exists" in str(e).lower():
        print(f"⚠️  Feature table ya existe, actualizando datos...")
        # Actualizar con merge
        fe.write_table(
            name=feature_table_name,
            df=features_df,
            mode="merge"
        )
        print(f"✅ Feature table actualizada: {feature_table_name}")
    else:
        raise e

# Verificar la tabla
feature_table = spark.table(feature_table_name)
print(f"\n📊 Total de registros: {feature_table.count():,}")
print(f"📊 Total de features: {len(feature_table.columns)}")

---

## 4️⃣ Experimentación con MLflow

**🔬 MLflow Tracking:**
- Registra automáticamente parámetros, métricas y artefactos
- Compara múltiples experimentos
- Reproduce resultados

In [ ]:
# Preparar datos para entrenamiento
training_data = feature_table.toPandas()

# Separar features y target
feature_columns = [
    'age', 'tenure_months', 'monthly_charges', 'monthly_minutes', 
    'data_usage_gb', 'support_tickets', 'avg_monthly_spend',
    'daily_data_usage', 'tickets_per_month', 'is_dsl', 'is_fiber',
    'is_monthly_contract', 'is_yearly_contract', 'is_electronic_check',
    'has_phone_service', 'has_streaming'
]

X = training_data[feature_columns]
y = training_data['churn']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Datos preparados para entrenamiento")
print(f"   Training set: {len(X_train):,} samples")
print(f"   Test set: {len(X_test):,} samples")
print(f"   Features: {len(feature_columns)}")

In [ ]:
# Función helper para entrenar y evaluar modelos
def train_and_evaluate_model(model, model_name, X_train, X_test, y_train, y_test, params=None):
    """
    Entrena un modelo y registra métricas en MLflow
    """
    with mlflow.start_run(run_name=model_name) as run:
        
        # Activar autolog para el framework correspondiente
        mlflow.sklearn.autolog(log_models=False)  # Registraremos el modelo manualmente
        
        # Entrenar modelo
        print(f"\n🚀 Entrenando {model_name}...")
        model.fit(X_train, y_train)
        
        # Predicciones
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Métricas
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        
        # Log métricas adicionales
        mlflow.log_metrics({
            "test_accuracy": accuracy,
            "test_precision": precision,
            "test_recall": recall,
            "test_f1_score": f1,
            "test_roc_auc": roc_auc
        })
        
        # Log parámetros del modelo
        if params:
            mlflow.log_params(params)
        
        # Log confusion matrix como artefacto
        cm = confusion_matrix(y_test, y_pred)
        cm_df = pd.DataFrame(
            cm, 
            index=['Actual No Churn', 'Actual Churn'],
            columns=['Predicted No Churn', 'Predicted Churn']
        )
        cm_df.to_csv('/tmp/confusion_matrix.csv')
        mlflow.log_artifact('/tmp/confusion_matrix.csv')
        
        # Feature importance (si está disponible)
        if hasattr(model, 'feature_importances_'):
            feature_importance = pd.DataFrame({
                'feature': feature_columns,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            feature_importance.to_csv('/tmp/feature_importance.csv', index=False)
            mlflow.log_artifact('/tmp/feature_importance.csv')
        
        # Registrar modelo con signature
        signature = infer_signature(X_train, y_pred_proba)
        
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="model",
            signature=signature,
            registered_model_name=f"telco_churn_{model_name.lower().replace(' ', '_')}"
        )
        
        # Imprimir resultados
        print(f"\n📊 Resultados de {model_name}:")
        print(f"   Accuracy:  {accuracy:.4f}")
        print(f"   Precision: {precision:.4f}")
        print(f"   Recall:    {recall:.4f}")
        print(f"   F1-Score:  {f1:.4f}")
        print(f"   ROC-AUC:   {roc_auc:.4f}")
        print(f"\n   Confusion Matrix:")
        print(cm_df)
        
        return run.info.run_id, model

In [ ]:
# EXPERIMENTO 1: Logistic Regression (baseline)
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_params = {
    'max_iter': 1000,
    'solver': 'lbfgs'
}

lr_run_id, lr_trained = train_and_evaluate_model(
    lr_model, 
    "Logistic Regression", 
    X_train, X_test, y_train, y_test,
    params=lr_params
)

In [ ]:
# EXPERIMENTO 2: Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

rf_params = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 5
}

rf_run_id, rf_trained = train_and_evaluate_model(
    rf_model,
    "Random Forest",
    X_train, X_test, y_train, y_test,
    params=rf_params
)

In [ ]:
# EXPERIMENTO 3: Gradient Boosting
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)

gb_params = {
    'n_estimators': 100,
    'learning_rate': 0.1,
    'max_depth': 5
}

gb_run_id, gb_trained = train_and_evaluate_model(
    gb_model,
    "Gradient Boosting",
    X_train, X_test, y_train, y_test,
    params=gb_params
)

In [ ]:
# Comparar todos los modelos
print("\n" + "="*60)
print("📊 COMPARACIÓN DE MODELOS")
print("="*60)

# Obtener runs del experimento
experiment = mlflow.get_experiment_by_name(experiment_name)
runs_df = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.test_roc_auc DESC"]
)

# Mostrar comparación
comparison_cols = [
    'run_id', 
    'tags.mlflow.runName',
    'metrics.test_accuracy',
    'metrics.test_precision',
    'metrics.test_recall',
    'metrics.test_f1_score',
    'metrics.test_roc_auc'
]

display(runs_df[comparison_cols].head())

best_model_name = runs_df.iloc[0]['tags.mlflow.runName']
best_auc = runs_df.iloc[0]['metrics.test_roc_auc']
print(f"\n🏆 Mejor modelo: {best_model_name} (ROC-AUC: {best_auc:.4f})")

---

## 5️⃣ Model Registry: Gestión del Ciclo de Vida

**🗂️ Stages del Model Registry:**
- **None**: Modelo registrado pero no promovido
- **Staging**: Modelo en testing/QA
- **Production**: Modelo en producción activo
- **Archived**: Modelo deprecado

In [ ]:
# Obtener el mejor modelo del experimento
best_run = runs_df.iloc[0]
best_run_id = best_run['run_id']
best_model_name = best_run['tags.mlflow.runName']

print(f"🏆 Promoviendo modelo a Staging: {best_model_name}")
print(f"   Run ID: {best_run_id}")

# Obtener el modelo URI
model_uri = f"runs:/{best_run_id}/model"
registered_model_name = "telco_churn_production"

# Registrar el modelo (si no existe)
try:
    model_details = mlflow.register_model(
        model_uri=model_uri,
        name=registered_model_name
    )
    model_version = model_details.version
    print(f"✅ Modelo registrado como versión {model_version}")
except Exception as e:
    print(f"⚠️  Error registrando modelo: {e}")
    # Si el modelo ya existe, obtener la última versión
    client = mlflow.MlflowClient()
    latest_versions = client.get_latest_versions(registered_model_name)
    model_version = latest_versions[0].version if latest_versions else 1

In [ ]:
# Transicionar modelo a Staging
client = mlflow.MlflowClient()

client.transition_model_version_stage(
    name=registered_model_name,
    version=model_version,
    stage="Staging",
    archive_existing_versions=True  # Archivar versiones anteriores en Staging
)

print(f"✅ Modelo transicionado a Staging")

# Agregar descripción y tags
client.update_model_version(
    name=registered_model_name,
    version=model_version,
    description=f"Modelo {best_model_name} entrenado el {datetime.now().strftime('%Y-%m-%d')}. ROC-AUC: {best_auc:.4f}"
)

client.set_model_version_tag(
    name=registered_model_name,
    version=model_version,
    key="model_type",
    value=best_model_name
)

client.set_model_version_tag(
    name=registered_model_name,
    version=model_version,
    key="training_date",
    value=datetime.now().strftime('%Y-%m-%d')
)

print("✅ Metadata agregada al modelo")

In [ ]:
# Simular validación en Staging (normalmente harías pruebas A/B, shadow mode, etc.)
print("\n🧪 VALIDACIÓN EN STAGING")
print("="*60)

# Cargar modelo desde Staging
model_staging_uri = f"models:/{registered_model_name}/Staging"
loaded_model = mlflow.sklearn.load_model(model_staging_uri)

# Validar en un conjunto de validación
y_val_pred_proba = loaded_model.predict_proba(X_test)[:, 1]
y_val_pred = loaded_model.predict(X_test)

val_accuracy = accuracy_score(y_test, y_val_pred)
val_roc_auc = roc_auc_score(y_test, y_val_pred_proba)

print(f"✅ Validación completada")
print(f"   Accuracy: {val_accuracy:.4f}")
print(f"   ROC-AUC: {val_roc_auc:.4f}")

# Criterio de promoción a producción
MIN_ROC_AUC_THRESHOLD = 0.70

if val_roc_auc >= MIN_ROC_AUC_THRESHOLD:
    print(f"\n✅ Modelo cumple criterios (ROC-AUC >= {MIN_ROC_AUC_THRESHOLD})")
    print("   Promoviendo a PRODUCTION...")
    
    client.transition_model_version_stage(
        name=registered_model_name,
        version=model_version,
        stage="Production",
        archive_existing_versions=True
    )
    
    print(f"\n🚀 Modelo en PRODUCTION (versión {model_version})")
else:
    print(f"\n❌ Modelo NO cumple criterios (ROC-AUC < {MIN_ROC_AUC_THRESHOLD})")
    print("   Se requiere más entrenamiento o ajuste de hiperparámetros")

---

## 6️⃣ Inferencia y Predicciones

**🔮 Modos de Inferencia:**
- **Batch**: Predicciones programadas sobre conjuntos grandes
- **Real-time**: API endpoint para predicciones individuales
- **Streaming**: Predicciones sobre datos en tiempo real

In [ ]:
# INFERENCIA BATCH: Predecir sobre nuevos clientes

# Simular nuevos clientes (en producción vendrían de tu pipeline de datos)
new_customers = pd.DataFrame({
    'customer_id': ['NEW_001', 'NEW_002', 'NEW_003'],
    'age': [25, 45, 60],
    'tenure_months': [3, 24, 48],
    'monthly_charges': [85.0, 65.0, 45.0],
    'monthly_minutes': [500, 1200, 800],
    'data_usage_gb': [15.0, 8.0, 3.0],
    'support_tickets': [5, 1, 0],
    'avg_monthly_spend': [28.33, 2.71, 0.94],
    'daily_data_usage': [0.5, 0.27, 0.1],
    'tickets_per_month': [1.67, 0.04, 0.0],
    'is_dsl': [0, 1, 0],
    'is_fiber': [1, 0, 0],
    'is_monthly_contract': [1, 0, 0],
    'is_yearly_contract': [0, 1, 1],
    'is_electronic_check': [1, 0, 0],
    'has_phone_service': [1, 1, 1],
    'has_streaming': [1, 1, 0]
})

print("🆕 Nuevos clientes para predecir:")
display(new_customers[['customer_id', 'age', 'tenure_months', 'monthly_charges']])

In [ ]:
# Cargar modelo de producción
model_prod_uri = f"models:/{registered_model_name}/Production"
production_model = mlflow.sklearn.load_model(model_prod_uri)

# Hacer predicciones
new_customers_features = new_customers[feature_columns]
churn_predictions = production_model.predict(new_customers_features)
churn_probabilities = production_model.predict_proba(new_customers_features)[:, 1]

# Crear DataFrame con resultados
predictions_df = pd.DataFrame({
    'customer_id': new_customers['customer_id'],
    'churn_prediction': churn_predictions,
    'churn_probability': churn_probabilities,
    'risk_level': pd.cut(
        churn_probabilities,
        bins=[0, 0.3, 0.7, 1.0],
        labels=['Bajo', 'Medio', 'Alto']
    )
})

print("\n🔮 PREDICCIONES DE CHURN")
print("="*60)
display(predictions_df)

# Guardar predicciones (en producción irían a una tabla Delta)
predictions_table_name = f"{database_name}.churn_predictions"
spark_predictions = spark.createDataFrame(predictions_df)
spark_predictions = spark_predictions.withColumn("prediction_timestamp", F.current_timestamp())

spark_predictions.write.mode("append").saveAsTable(predictions_table_name)
print(f"\n✅ Predicciones guardadas en: {predictions_table_name}")

---

## 7️⃣ Monitoreo de Modelos en Producción

**📊 ¿Qué monitorear?**
- **Performance**: Métricas de negocio y ML
- **Data Drift**: Cambios en la distribución de features
- **Prediction Drift**: Cambios en las predicciones
- **Latencia**: Tiempo de respuesta
- **Volumen**: Número de predicciones

In [ ]:
# Simular monitoreo de performance en producción
# En la realidad, compararías predicciones con outcomes reales (cuando estén disponibles)

def calculate_monitoring_metrics(predictions_df, actuals_df=None):
    """
    Calcula métricas de monitoreo para el modelo en producción
    """
    metrics = {}
    
    # Distribución de predicciones
    metrics['avg_churn_probability'] = predictions_df['churn_probability'].mean()
    metrics['predicted_churn_rate'] = predictions_df['churn_prediction'].mean()
    
    # Distribución de riesgo
    risk_distribution = predictions_df['risk_level'].value_counts(normalize=True)
    metrics['high_risk_percentage'] = risk_distribution.get('Alto', 0)
    metrics['medium_risk_percentage'] = risk_distribution.get('Medio', 0)
    metrics['low_risk_percentage'] = risk_distribution.get('Bajo', 0)
    
    # Si tenemos actuals, calcular performance real
    if actuals_df is not None:
        merged = predictions_df.merge(actuals_df, on='customer_id')
        metrics['actual_accuracy'] = accuracy_score(
            merged['actual_churn'], 
            merged['churn_prediction']
        )
        metrics['actual_roc_auc'] = roc_auc_score(
            merged['actual_churn'],
            merged['churn_probability']
        )
    
    return metrics

# Calcular métricas
monitoring_metrics = calculate_monitoring_metrics(predictions_df)

print("\n📈 MÉTRICAS DE MONITOREO")
print("="*60)
for metric_name, metric_value in monitoring_metrics.items():
    print(f"{metric_name:.<40} {metric_value:.4f}")

# Log métricas de monitoreo a MLflow
with mlflow.start_run(run_name="production_monitoring") as run:
    mlflow.log_metrics(monitoring_metrics)
    mlflow.log_param("monitoring_date", datetime.now().strftime('%Y-%m-%d'))
    mlflow.log_param("model_version", model_version)
    
print("\n✅ Métricas de monitoreo registradas en MLflow")

In [ ]:
# DATA DRIFT MONITORING
# Comparar distribución de features entre entrenamiento y producción

def check_data_drift(training_data, production_data, threshold=0.1):
    """
    Detecta drift comparando estadísticas entre training y producción
    """
    drift_report = []
    
    for feature in feature_columns:
        # Calcular estadísticas
        train_mean = training_data[feature].mean()
        train_std = training_data[feature].std()
        
        prod_mean = production_data[feature].mean()
        prod_std = production_data[feature].std()
        
        # Calcular diferencia relativa
        mean_diff = abs(prod_mean - train_mean) / (train_mean + 1e-10)
        std_diff = abs(prod_std - train_std) / (train_std + 1e-10)
        
        # Detectar drift
        has_drift = (mean_diff > threshold) or (std_diff > threshold)
        
        drift_report.append({
            'feature': feature,
            'train_mean': train_mean,
            'prod_mean': prod_mean,
            'mean_diff_%': mean_diff * 100,
            'has_drift': has_drift
        })
    
    return pd.DataFrame(drift_report)

# Detectar drift
drift_df = check_data_drift(X_train, new_customers_features, threshold=0.15)

print("\n🚨 DATA DRIFT ANALYSIS")
print("="*60)
print(f"Features con drift detectado: {drift_df['has_drift'].sum()}")
print("\nTop 5 features con mayor diferencia:")
display(drift_df.nlargest(5, 'mean_diff_%')[['feature', 'mean_diff_%', 'has_drift']])

# Si hay drift significativo, alertar
if drift_df['has_drift'].sum() > len(feature_columns) * 0.3:
    print("\n⚠️  ALERTA: Drift significativo detectado (>30% de features)")
    print("   Considerar reentrenamiento del modelo")
else:
    print("\n✅ Drift dentro de límites aceptables")

---

## 8️⃣ CI/CD y Automatización

**🔄 Pipeline Automatizado:**
```
1. Trigger (schedule o evento)
   ↓
2. Data Pipeline (actualizar Feature Store)
   ↓
3. Training Pipeline (entrenar modelos)
   ↓
4. Evaluation (validar performance)
   ↓
5. Staging (deploy a staging)
   ↓
6. Testing (A/B testing, shadow mode)
   ↓
7. Production (deploy a producción)
   ↓
8. Monitoring (alertas y dashboards)
```

In [ ]:
# EJEMPLO: Función de reentrenamiento automatizado

def automated_retraining_pipeline(
    feature_table_name,
    model_name,
    min_roc_auc_threshold=0.70,
    retrain_if_drift=True
):
    """
    Pipeline automatizado de reentrenamiento
    
    Esta función se ejecutaría en un Databricks Job programado
    """
    
    print("🔄 INICIANDO PIPELINE DE REENTRENAMIENTO")
    print("="*60)
    
    # 1. Cargar datos actualizados del Feature Store
    print("\n1️⃣ Cargando datos del Feature Store...")
    feature_data = spark.table(feature_table_name).toPandas()
    print(f"   ✅ {len(feature_data):,} registros cargados")
    
    # 2. Preparar datos
    print("\n2️⃣ Preparando datos...")
    X = feature_data[feature_columns]
    y = feature_data['churn']
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print(f"   ✅ Train: {len(X_train):,} | Test: {len(X_test):,}")
    
    # 3. Entrenar modelo
    print("\n3️⃣ Entrenando modelo...")
    with mlflow.start_run(run_name="automated_retraining") as run:
        # Usar el mejor algoritmo del experimento anterior
        model = GradientBoostingClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
        
        mlflow.sklearn.autolog()
        model.fit(X_train, y_train)
        
        # Evaluar
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        
        mlflow.log_metric("test_roc_auc", roc_auc)
        mlflow.log_param("retraining_date", datetime.now().strftime('%Y-%m-%d'))
        
        print(f"   ✅ ROC-AUC: {roc_auc:.4f}")
        
        # 4. Validar performance
        print("\n4️⃣ Validando performance...")
        if roc_auc >= min_roc_auc_threshold:
            print(f"   ✅ Modelo cumple threshold (>={min_roc_auc_threshold})")
            
            # 5. Registrar y promover modelo
            print("\n5️⃣ Registrando modelo...")
            model_uri = f"runs:/{run.info.run_id}/model"
            model_details = mlflow.register_model(
                model_uri=model_uri,
                name=model_name
            )
            
            print(f"   ✅ Modelo registrado como versión {model_details.version}")
            
            # Transicionar a Staging
            client = mlflow.MlflowClient()
            client.transition_model_version_stage(
                name=model_name,
                version=model_details.version,
                stage="Staging"
            )
            
            print("   ✅ Modelo promovido a Staging")
            print("\n   ⚠️  Requiere validación manual antes de Production")
            
            return True, model_details.version, roc_auc
        else:
            print(f"   ❌ Modelo NO cumple threshold (<{min_roc_auc_threshold})")
            print("   ⚠️  Reentrenamiento fallido - revisar datos y features")
            return False, None, roc_auc

# Ejecutar pipeline (en producción esto sería un Databricks Job)
success, new_version, new_roc_auc = automated_retraining_pipeline(
    feature_table_name=feature_table_name,
    model_name=registered_model_name,
    min_roc_auc_threshold=0.70
)

if success:
    print(f"\n🎉 REENTRENAMIENTO EXITOSO")
    print(f"   Nueva versión: {new_version}")
    print(f"   ROC-AUC: {new_roc_auc:.4f}")
else:
    print(f"\n⚠️  REENTRENAMIENTO REQUIERE ATENCIÓN")

---

## 9️⃣ Best Practices y Recomendaciones

### 🎯 Organización del Proyecto

```
mlops-project/
├── notebooks/
│   ├── 01_data_exploration.py
│   ├── 02_feature_engineering.py
│   ├── 03_model_training.py
│   └── 04_model_evaluation.py
├── src/
│   ├── features/
│   │   └── feature_engineering.py
│   ├── models/
│   │   ├── train.py
│   │   └── predict.py
│   └── utils/
│       └── helpers.py
├── tests/
│   ├── test_features.py
│   └── test_models.py
├── configs/
│   ├── training_config.yaml
│   └── deployment_config.yaml
└── workflows/
    ├── training_pipeline.py
    └── deployment_pipeline.py
```

### 📋 Checklist de MLOps

#### Desarrollo
- ✅ Experimentos rastreados en MLflow
- ✅ Features versionadas en Feature Store
- ✅ Código versionado en Git
- ✅ Tests unitarios para features y modelos

#### Deployment
- ✅ Modelos registrados en Model Registry
- ✅ Validación en Staging antes de Production
- ✅ CI/CD pipeline automatizado
- ✅ Rollback plan documentado

#### Monitoreo
- ✅ Métricas de performance rastreadas
- ✅ Data drift detectado automáticamente
- ✅ Alertas configuradas
- ✅ Dashboard de monitoreo

#### Gobernanza
- ✅ Documentación de modelos
- ✅ Lineage de datos y modelos
- ✅ Políticas de retención
- ✅ Auditoría de cambios

### 🔧 Configuración de Jobs en Databricks

**Job de Reentrenamiento Mensual:**
```python
# Databricks Job Configuration
{
  "name": "churn_model_retraining",
  "schedule": {
    "quartz_cron_expression": "0 0 1 * * ?",  # 1er día de cada mes
    "timezone_id": "America/New_York"
  },
  "tasks": [
    {
      "task_key": "update_features",
      "notebook_task": {
        "notebook_path": "/Workflows/feature_engineering"
      }
    },
    {
      "task_key": "train_model",
      "depends_on": [{"task_key": "update_features"}],
      "notebook_task": {
        "notebook_path": "/Workflows/model_training"
      }
    },
    {
      "task_key": "validate_model",
      "depends_on": [{"task_key": "train_model"}],
      "notebook_task": {
        "notebook_path": "/Workflows/model_validation"
      }
    }
  ]
}
```

### 📚 Recursos Adicionales

- [Databricks MLOps Guide](https://docs.databricks.com/mlflow/index.html)
- [MLflow Documentation](https://mlflow.org/docs/latest/index.html)
- [Feature Store Best Practices](https://docs.databricks.com/machine-learning/feature-store/index.html)
- [Model Registry Guide](https://docs.databricks.com/machine-learning/model-registry/index.html)

---

## 🎓 Ejercicios Prácticos

### Ejercicio 1: Mejorar el Modelo
1. Crea nuevas features derivadas (ej: ratio de datos vs minutos)
2. Prueba XGBoost o LightGBM
3. Implementa hyperparameter tuning con Hyperopt
4. Compara resultados en MLflow

### Ejercicio 2: Implementar A/B Testing
1. Deploy dos versiones del modelo en Staging
2. Divide tráfico 50/50
3. Compara métricas de negocio
4. Promover el ganador a Production

### Ejercicio 3: Dashboard de Monitoreo
1. Crea una tabla de métricas históricas
2. Visualiza trends de performance
3. Detecta anomalías en predicciones
4. Configura alertas automáticas

### Ejercicio 4: CI/CD Pipeline
1. Configura un Databricks Workflow
2. Automatiza feature engineering
3. Automatiza training y validation
4. Implementa promoción automática a Staging

---

## 📝 Notas Finales

Este notebook cubre el ciclo completo de MLOps:

1. ✅ **Feature Engineering** con Feature Store
2. ✅ **Experimentación** con MLflow Tracking
3. ✅ **Versionado** con Model Registry
4. ✅ **Deployment** en Staging y Production
5. ✅ **Inferencia** batch y real-time
6. ✅ **Monitoreo** de performance y drift
7. ✅ **Automatización** con pipelines

### 🎯 Próximos Pasos

- Implementar serving con **Databricks Model Serving**
- Integrar con sistemas externos (CRM, Marketing)
- Configurar **Unity Catalog** para gobernanza
- Implementar **drift detection** avanzado
- Crear dashboards en **Databricks SQL**

---

**¡Felicidades! Has completado el workshop de MLOps con Databricks** 🎉